# Testing for the loss:

In [109]:
import numpy as np
from sklearn.cross_decomposition import CCA
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
from objectives import cca_loss
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle
import pickle
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import sklearn.model_selection as model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import pprint

In [110]:
U = np.random.random_sample(10000).reshape(250,40)
V = np.random.random_sample(10000).reshape(250,40)

In [129]:
# Get the first n components:
n_comps = 10
cca = CCA(n_components=n_comps)
U_c_1, V_c_1 = cca.fit_transform(U, V) # Fits and transforms
print(U_c_1.shape, V_c_1.shape)
result1 = np.corrcoef(U_c_1.T, V_c_1.T).diagonal(offset=n_comps) # find how similar the respective components are

l_cca = linear_cca()
l_cca.fit(U,V,n_comps) # Fits, saves the transformation materices (= the weights the matrices should be multiplied with) to obj
U_c_2, V_c_2 = np.matmul(U, l_cca.w[0]), np.matmul(V, l_cca.w[1]) # get the transformed matrices by multiplying
print(U_c_2.shape, V_c_2.shape)
result2 = np.corrcoef(U_c_2.T, V_c_2.T).diagonal(offset=n_comps) # find how similar the respective components are


(250, 10) (250, 10)
(250, 10) (250, 10)


In [130]:
print("result1 shape:",result1.shape)
print("result2 shape:",result2.shape)
print("result1:", result1)
print("result2:", result2)
print(sum(result1))
print(sum(result2))
print("SKL cca score:",cca.score(U, V))

result1 shape: (10,)
result2 shape: (10,)
result1: [0.73237705 0.7199459  0.71436048 0.67332589 0.65441498 0.61007048
 0.59099732 0.55817217 0.5544709  0.53278441]
result2: [0.73238195 0.71996561 0.71433531 0.67333231 0.65440846 0.61007296
 0.59099479 0.55819154 0.55445301 0.53278184]
6.3409195866105605
6.340917781691015
SKL cca score: 0.0663523194219422


In [113]:
c_loss = cca_loss(10, True, torch.device('cpu'))
correlation = c_loss.loss(torch.from_numpy(U),torch.from_numpy(V))
print(correlation)

tensor(-16.3168)
